# Decision Trees
[![Run in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/bobmh43/handson_ml/blob/master/notebooks/ch6_decision_trees.ipynb)

# End of chapter exercises:

*1. What is the approximate depth of a Decision Tree trained (without restrictions) on a training set with 1 million instances?*

Assume we are using the CART training algorithm. Each split divides the training samples into two subsets of roughly the same size. Thus, the depth would be log_2(m) = 20.

**2. Is a node’s Gini impurity generally lower or greater than its parent’s? Is it generally lower/greater, or always lower/greater?**

A node's Gini impurity is usually lower than its parents but not always. It is usually lower as the split minimizes the weighted sum of the impurities of the two resulting children nodes. But it is not always lower. For example, a [19, 1]-node could be split into a [10, 0]-node and a [9,1]-node. The latter node has a higher impurity than the original node.


*3. If a Decision Tree is overfitting the training set, is it a good idea to try decreasing max_depth?*

Yes. Reducing `max_depth` makes the model less complex, regularizing it.


*4. If a Decision Tree is underfitting the training set, is it a good idea to try scaling the input features?*

A decision tree is not affected by the scales of the features of the training instances. Rescaling the features only alters the thresholds of the tree but does not alter the tree's performance.


*5. If it takes one hour to train a Decision Tree on a training set containing 1 million instances, roughly how much time will it take to train another Decision Tree on a training set containing 10 million instances?*

The training algorithm has a time complexity of O(nmlogm). So 10 million instances will take n(10^7)log(10^7) / n(10^6)log(10^6) = 11.7 hours.


*6. If your training set contains 100,000 instances, will setting presort=True speed up training?*

No, `presort` would not. The overhead of presorting the training instances outweighs the reduction in time in finding the right threshold once the number of instances exceeds a few thousand.



In [ ]:
import numpy as np
from scipy.stats import mode
import sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, ShuffleSplit, RandomizedSearchCV
from sklearn.metrics import accuracy_score

# Plant a tree

In [ ]:
# prepare some moon data
X, y = sklearn.datasets.make_moons(n_samples=10000, noise=0.4, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=43)

In [ ]:
# create a model (no StandardScaler and Pipeline needed, no PolynomialFeatures needed either)
tree_clf = DecisionTreeClassifier(random_state=41)

In [ ]:
# search for the best hyperparameters
param_distrib = {
    "max_leaf_nodes": np.arange(2, 100),
    "max_depth": np.arange(2, 20),
    "min_samples_split": np.arange(2, 6)
}
rscv = RandomizedSearchCV(tree_clf, param_distrib, n_iter=500, cv=5)
rscv.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=41),
                   n_iter=500,
                   param_distributions={'max_depth': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19]),
                                        'max_leaf_nodes': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52,
       53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69,
       70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86,
       87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]),
                                        'min_samples_split': array([2, 3, 4, 5])})

In [ ]:
print("Best estimator: ", repr(rscv.best_estimator_))
print("Mean val accuracy score of the best estimator:", rscv.best_score_)
print("Test accuracy: ", rscv.best_estimator_.score(X_test, y_test))
# reminder again, cross_val_score is used to estimate the generalization error and is a better estimate than just one train-val split
# cross_val_predict should not be used to estimate the generalization error. It just gives "honest" out-of-sample predictions and should only be used for diagnostic purposes such as confusion matrices, ROC curves and calibration curves.

Best estimator:  DecisionTreeClassifier(max_depth=np.int64(7), max_leaf_nodes=np.int64(28),
                       min_samples_split=np.int64(5), random_state=41)
Mean val accuracy score of the best estimator: 0.861125
Test accuracy:  0.853


# Grow a forest

In [ ]:
# create a forest
# using the best parameters from before, create 1000 such trees.
n_trees = 1000
forest = [sklearn.base.clone(rscv.best_estimator_) for i in range(n_trees)] # these are unfitted

In [ ]:
# train them on subsets of the training data (so that they have quite some variance)
shuffle_split = ShuffleSplit(n_splits=n_trees, train_size=100, random_state=47)
val_scores = [None] * n_trees
for idx, (tree, (train_indices, test_indices)) in enumerate(zip(forest, shuffle_split.split(X_train))):
  val_scores[idx] = tree.fit(X_train[train_indices], y_train[train_indices]) \
                        .score(X_train[test_indices], y_train[test_indices])
print("mean accuracy of a tree in the forest: ", np.mean(val_scores))

mean accuracy of a tree in the forest:  0.798409746835443


In [ ]:
# majority voting
def predict(forest, X):
  # a grid of (n_trees, n_instances)
  prediction_grid = list(map(lambda tree: tree.predict(X), forest))
  ModeResult = mode(prediction_grid, axis=0)
  return ModeResult.mode

print("training accuracy of the whole forest: ", accuracy_score(y_train, predict(forest, X_train)))
print("test accuracy of the whole forest: ", accuracy_score(y_test, predict(forest, X_test)))

training accuracy of the whole forest:  0.867625
test accuracy of the whole forest:  0.8625


An improvement of 0.9%!